In [1]:
# -*- coding: UTF-8 -*-
import time
import pandas as pd
import pickle
import os
from datetime import datetime,timedelta
from sklearn.ensemble import GradientBoostingClassifier
import random
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from operator import itemgetter
import operator
from sklearn.model_selection import KFold
import scipy as sp
import gc
import lightgbm as lgb
from collections import Counter
from collections import defaultdict
import csv
#dl
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
# #读取训练集
# d28 = pd.read_csv('../280000.csv')
# d29 = pd.read_csv('../290000.csv')

# traindata = d28.copy()
# print len(traindata)

# #读取测试集
# d30 = pd.read_csv('../300000.csv')
# testdata = pd.concat([d29, d30])

traindata = pd.read_csv('trainNN.csv')
print len(traindata)

#获取test label
test_label_pd = pd.read_csv('trainLabel.csv')
test_label = test_label_pd['label']
print len(test_label)

2700288
2700288


In [3]:
#获取app描述
app_category = pd.read_csv('app200_category.csv')

#获取训练app
train_app_copy = traindata[['app_id']].copy()
app_des = app_category[['app_id', 'des']].copy()
train_app = pd.merge(train_app_copy, app_des, how = 'left', on = 'app_id')
print len(train_app)
#911479
#895269
#893540

2700288


In [4]:
#获取train中出现的userid
user_map = defaultdict(int)
for i in traindata['user_id']:
    user_map[int(i)] = 1

In [1]:
#获取train app matrix
def get_app_matrix(train_app):
    app_matrix = []
    app_set = []
    app_pos = defaultdict(int)

    for i in train_app['des']:
        str1 = i.split(',')
        app_set += str1
    app_set = list(set(app_set))
    app_set = app_set[0:1000]
    print len(app_set)

    #记录每一个元素的位置
    s_vec = 0
    for i in app_set:
        app_pos[i] = s_vec
        s_vec += 1

    len_vec = len(app_set)
    print len_vec

    for i in train_app['des']:
        templist = []
        for ln in range(0, len_vec):
            templist.append(0)
        str1 = i.split(',')
        for j in str1:
            templist[app_pos[j]] = 1
        app_matrix.append(templist)


    app_matrix = np.array(app_matrix)
    return app_matrix

In [5]:
app_matrix = get_app_matrix(train_app)
print app_matrix.shape
np.save("app_matrix.npy", app_matrix) 

1000
1000
(2700288, 1000)


In [5]:
#取出query
query_1000 = defaultdict(int)
query_temp_list = []
query_map = {}
with open('dataset_query/query.csv', 'rb') as csvfile:
    sdata = csv.DictReader(csvfile)
    for rec in sdata:
        if (user_map[int(rec['user_id'])] == 0):
            continue
        temp_query = rec['query'].split(',')
        if (not query_map.has_key(rec['user_id'])):
            query_map[rec['user_id']] = []

        query_map[rec['user_id']].append(temp_query)
        
        for i in temp_query:
            query_1000[i] += 1
            
test_query_num = []
for i in query_1000.keys():
    test_query_num.append(query_1000[i])
test_query_num.sort(reverse=True)

min_query_1000 = test_query_num[999]

#获取1000个词
word_query = []
for i in query_1000.keys():
    if (query_1000[i] >= min_query_1000):
        word_query.append(i)
print len(word_query)

len_query_1000 = len(word_query)

#获取query word 位置
query_pos = defaultdict(int)
query_l = 0
for i in word_query:
    query_pos[i] = query_l
    query_l += 1
    
#获取query矩阵 
query_matrix = []
num = 0
for i in traindata['user_id']:
    mx = []
    num += 1
    if (num %100000 == 0):
        print num
        
    temp_mx = []
    for tm in range(0, len_query_1000):
        temp_mx.append(0)
    if (not query_map.has_key(str(i))):
        query_matrix.append(temp_mx)
        continue
        
#     for j in query_map[str(i)]:
#         temp_j = []
#         for lj in range(0, len_query_1000):
#             temp_j.append(0)
#         for wj in j:
#             temp_j[query_pos[wj]] = 1
            
#         temp_mx = list(map(lambda x: x[0] + x[1], zip(temp_mx, temp_j)))  
    j = random.sample(query_map[str(i)], 1)[0]
    temp_j = []
    for lj in range(0, len_query_1000):
        temp_j.append(0)
    for wj in j:
        temp_j[query_pos[wj]] = 1
            
    query_matrix.append(temp_j)
    
query_matrix = np.array(query_matrix)

np.save("query_matrix.npy", query_matrix) 

print query_matrix.shape

1000
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
(2700288, 1000)


In [5]:
#取出feed前1000个词
feed_1000 = defaultdict(int)
feed_temp_list = []
feed_map = {}
with open('dataset_newsfeed/newsfeed2.csv', 'rb') as csvfile:
    sdata = csv.DictReader(csvfile)
    for rec in sdata:
        if (user_map[int(rec['user_id'])] == 0):
            continue
        temp_feed = rec['news'].split(',')
        if (not feed_map.has_key(rec['user_id'])):
            feed_map[rec['user_id']] = []

        feed_map[rec['user_id']].append(temp_feed)
        
        for i in temp_feed:
            feed_1000[i] += 1
            
test_feed_num = []
for i in feed_1000.keys():
    test_feed_num.append(feed_1000[i])
test_feed_num.sort(reverse=True)

min_feed_1000 = test_feed_num[999]

#获取1000个词
word_feed = []
for i in feed_1000.keys():
    if (feed_1000[i] >= min_feed_1000):
        word_feed.append(i)
print len(word_feed)

len_feed_1000 = len(word_feed)

#获取feed word 位置
feed_pos = defaultdict(int)
feed_l = 0
for i in word_feed:
    feed_pos[i] = feed_l
    feed_l += 1
    
#获取feed矩阵 
feed_matrix = []
num = 0
for i in traindata['user_id']:
    mx = []
    num += 1
    if (num %1000000 == 0):
        print num
        
    temp_mx = []
    for tm in range(0, len_feed_1000):
        temp_mx.append(0)
    if (not feed_map.has_key(str(i))):
        feed_matrix.append(temp_mx)
        continue
        
    for j in feed_map[str(i)]:
        temp_j = []
        for lj in range(0, len_feed_1000):
            temp_j.append(0)
        for wj in j:
            temp_j[feed_pos[wj]] = 1
            
        temp_mx = list(map(lambda x: x[0] + x[1], zip(temp_mx, temp_j)))  
    
    feed_matrix.append(temp_mx)
    
feed_matrix = np.array(feed_matrix)

np.save("feed_matrix.npy", feed_matrix) 

print feed_matrix.shape
    

1000
1000000
2000000
(2700288, 1000)


In [6]:
#取shopping前1000个词
shop_1000 = defaultdict(int)
shop_temp_list = []
shop_map = {}
with open('shopping_data2.csv', 'rb') as csvfile:
    sdata = csv.DictReader(csvfile)
    num = 0
    for rec in sdata:
        num += 1
        if (num % 10000000 == 0):
            print num
        if (user_map[int(rec['user_id'])] == 0):
            continue
        temp_shopping = rec['shopping'].split(',')
        
        if (not shop_map.has_key(rec['user_id'])):
            shop_map[rec['user_id']] = []

        shop_map[rec['user_id']].append(temp_shopping)
        for i in temp_shopping:
            shop_1000[i] += 1
            
test_shop_num = []
for i in shop_1000.keys():
    test_shop_num.append(shop_1000[i])
test_shop_num.sort(reverse=True)

min_shop = test_shop_num[999]

#获取1000个词
word_shop = []
for i in shop_1000.keys():
    if (shop_1000[i] >= min_shop):
        word_shop.append(i)
print len(word_shop)

len_shop = len(word_shop)

#获取shop word 位置
shop_pos = defaultdict(int)
shop_l = 0
for i in word_shop:
    shop_pos[i] = shop_l
    shop_l += 1
    
#获取shopping矩阵 
shop_matrix = []
num = 0
for i in traindata['user_id']:
    mx = []
    num += 1
    if (num %1000000 == 0):
        print num
        
    temp_mx = []
    for tm in range(0, len_shop):
        temp_mx.append(0)
    if (not shop_map.has_key(str(i))):
        shop_matrix.append(temp_mx)
        continue
        
    for j in shop_map[str(i)]:
        temp_j = []
        for lj in range(0, len_shop):
            temp_j.append(0)
        for wj in j:
            temp_j[shop_pos[wj]] = 1
            
        temp_mx = list(map(lambda x: x[0] + x[1], zip(temp_mx, temp_j)))  
    
    shop_matrix.append(temp_mx)
    
shop_matrix = np.array(shop_matrix)
print shop_matrix.shape

np.save('shop_matrix.npy', shop_matrix)

10000000
1000
1000000
2000000
(2700288, 1000)


In [5]:
#加载训练测试数据
app_matrix = np.load('app_matrix.npy')
feed_matrix = np.load("feed_matrix.npy")
shop_matrix = np.load('shop_matrix.npy')
query_matrix = np.load('query_matrix.npy')
xgb_matrix = np.load('xgb_feature_float.npy')

print app_matrix.shape
print feed_matrix.shape
print shop_matrix.shape
print query_matrix.shape
print xgb_matrix.shape

(2700288, 1000)
(2700288, 1000)
(2700288, 1000)
(2700288, 1000)
(2700288, 48)


In [8]:
'''
define model
'''

NUM_UNITS = 2000
RES_DEPTH = 16
batch_size = 128
num_iterations = 10000001

def resblock(x):
    h = tf.contrib.layers.fully_connected(
        inputs=tf.contrib.layers.batch_norm(tf.nn.relu(x)),
        num_outputs=NUM_UNITS,
#         normalizer_fn=tf.contrib.layers.batch_norm,
#         activation_fn=tf.nn.relu,
    )
    y = tf.contrib.layers.fully_connected(
        inputs=tf.contrib.layers.batch_norm(tf.nn.relu(h)),
        num_outputs=NUM_UNITS,
#         normalizer_fn=tf.contrib.layers.batch_norm,
#         activation_fn=tf.nn.relu,
    )
    return x + y

def resnet(x, depth):
    res = tf.contrib.layers.fully_connected(
        #inputs=x, 
        inputs = tf.nn.dropout(x, 0.5),
        num_outputs=NUM_UNITS,
        normalizer_fn=tf.contrib.layers.batch_norm,
        activation_fn=tf.nn.relu,
    )

    for i in range(depth):
        res = resblock(res)
    return res

class Model:
    def __init__(self, sess, idim, odim, keep_rate, checkpoint_load, checkpoint_save):
        
        self.op_shopping = tf.placeholder(np.float32, [None, 1000])
        self.op_app = tf.placeholder(np.float32, [None, 1000])
        self.op_feed = tf.placeholder(np.float32, [None, 1000])
        self.op_query = tf.placeholder(np.float32, [None, 1000])
        self.op_xgb = tf.placeholder(np.float32, [None, 48])
        
        self.op_Y = tf.placeholder(np.int32, [None])
        self.keep_rate = tf.constant(keep_rate)
        self.op_global_step = tf.Variable(1, trainable=False, name='global_step')
        self.op_learning_rate = tf.train.exponential_decay(1e-4, self.op_global_step, 5000, 0.9) + 1e-5
        
        res = tf.concat([self.op_shopping, self.op_app, self.op_feed, self.op_query, self.op_xgb], 1)
        res = resnet(res, 4)

        h2 = tf.contrib.layers.fully_connected(
            inputs=res, 
            num_outputs=odim,
        )
        self.logits = h2

        self.op_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels = self.op_Y, logits = self.logits))
        
        optimizer = tf.train.MomentumOptimizer(self.op_learning_rate, 0.9)
        grads_and_vars = optimizer.compute_gradients(self.op_loss)
        grads = [grad for grad, var in grads_and_vars]
        clipped_grads, global_norm = tf.clip_by_global_norm(grads, 10)

        self.op_train = optimizer.apply_gradients(zip(clipped_grads, [var for grad, var in grads_and_vars]))

        self.op_output = tf.nn.softmax(self.logits)
        self.odim = odim
        self.checkpoint_load = checkpoint_load
        self.checkpoint_save = checkpoint_save
        
        self.saver = tf.train.Saver()
        if checkpoint_load == None:
            sess.run(tf.initialize_all_variables())
        else:
            self.saver.restore(sess, self.checkpoint_load)
        self.history = []
    
    def train(self, sess, X, Y, loss=False, learning_rate=None):
        ops = [self.op_train, self.op_loss]
        shopping, app, feed, query, xgb = X
        feed_dict = {
            self.op_shopping: shopping,
            self.op_app: app,
            self.op_feed: feed,
            self.op_query: query,
            self.op_xgb: xgb,
            self.op_Y: Y,
        }
        if learning_rate != None:
            feed_dict[self.op_learning_rate] = learning_rate
        
        ans = sess.run(
            ops,
            feed_dict=feed_dict,
        )
        self.history += [ans[1]]
        if len(self.history) % 1000 == 0:
            self.saver.save(sess, self.checkpoint_save, self.op_global_step)
        return ans[1:]
    
    def predict(self, sess, X):
        return sess.run(
            self.op_output,
            feed_dict={
                self.op_X: X,
                self.keep_rate: 1,
            }
        )
    
    def plot(self):
        figure()
        b = len(self.history) // 300 + 1
        n = len(self.history) // b * b
        x = np.mean(np.array(self.history[:n]).reshape((-1, b)), axis=1)
        plot(np.arange(len(x)) * b, np.minimum(x, log(self.odim)))
        show()
        

In [9]:
tf.reset_default_graph()
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)
model_cvr = Model(sess, 4048, 2, 1., None, "model-gender-2")

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [10]:
def get_features(row):
    Y = train_row[row]
    shop_x = shop_matrix[row]
    app_x = app_matrix[row]
    feed_x = feed_matrix[row]
    query_x = query_matrix[row]
    xgb_x = xgb_matrix[row]
    X = [shop_x, app_x, feed_x, query_x, xgb_x]
    return X, Y

#获取训练集每一行
train_row = []

for i in test_label:
    train_row.append(i)

In [39]:
print 911479 + 895269
#893540

1806748


In [11]:
'''
train
'''
def train(model):
    X = []
    Y = np.zeros((batch_size,), dtype=np.int32)

    for i in xrange(batch_size):
        #随机行
        rand_row = random.randint(1, 911479)
        
        x, y= get_features(rand_row - 1)
        X += [x]
        Y[i] = y 
    X = [np.vstack([x[y] for x in X]) for y in range(5)]

    has_loss = T % 100 == 0
    loss = model.train(sess, X, Y, has_loss, learning_rate=1e-3)
    if has_loss:
        print "#%d: loss = %.6f" % (T, loss[0])

for T in xrange(10000000):

    train(model_cvr)

#0: loss = 0.981204
#100: loss = 0.567364
#200: loss = 0.614913
#300: loss = 0.419695
#400: loss = 0.513266
#500: loss = 0.534415
#600: loss = 0.645295
#700: loss = 0.508196
#800: loss = 0.583906
#900: loss = 0.499742
#1000: loss = 0.611360
#1100: loss = 0.564563
#1200: loss = 0.430775
#1300: loss = 0.506156
#1400: loss = 0.459691
#1500: loss = 0.565612
#1600: loss = 0.502488
#1700: loss = 0.396105
#1800: loss = 0.429918
#1900: loss = 0.582383
#2000: loss = 0.596589
#2100: loss = 0.507341
#2200: loss = 0.415008
#2300: loss = 0.508158
#2400: loss = 0.460174
#2500: loss = 0.476030
#2600: loss = 0.617267
#2700: loss = 0.448216
#2800: loss = 0.414999
#2900: loss = 0.470626
#3000: loss = 0.441852
#3100: loss = 0.453750
#3200: loss = 0.539206
#3300: loss = 0.437571
#3400: loss = 0.522737
#3500: loss = 0.499800
#3600: loss = 0.486756
#3700: loss = 0.435853
#3800: loss = 0.368289
#3900: loss = 0.427188
#4000: loss = 0.372047
#4100: loss = 0.421583
#4200: loss = 0.455774
#4300: loss = 0.486210


KeyboardInterrupt: 

In [13]:
def get_features2(row):
    shop_x = shop_matrix[row]
    app_x = app_matrix[row]
    feed_x = feed_matrix[row]
    query_x = feed_matrix[row]
    xgb_x = xgb_matrix[row]
    X = [shop_x, app_x, feed_x, query_x, xgb_x]
    return X

In [14]:
#预测用户
def model_predict(model, sess, X):
    shopping, app, feed, query, xgb = X
    feed_dict = {
        model.op_shopping: shopping,
        model.op_app: app,
        model.op_feed: feed,
        model.op_query: query, 
        model.op_xgb: xgb,
    }
    return sess.run(
        model.op_output,
        feed_dict=feed_dict
    )
    

def predict(dev, batch):

    X = []
    
    for i in range(dev, batch):
        
        x= get_features2(i)
        X += [x]
        
    X = [np.vstack([x[y] for x in X]) for y in range(5)]

    return model_predict(model_cvr, sess, X)

In [15]:
#测试
#ans1008 = []
def cv(cev, dev):
    loss_cvr = 0.

    cnt = 0

    B = 128
    with open('ans1011_02_NN.csv', 'w') as fout:
        print >> fout, 'prob'
        pred_gender = predict(cev, dev)
        for i in range(cev, dev):
            cvr = test_label[i]
            #ans1008.append(pred_gender[i][cvr])
            print >> fout, '%.9f'%pred_gender[cnt][1] 
            loss_cvr -= np.log(pred_gender[cnt][cvr] + 1e-10)
            cnt += 1
            if cnt % 100000 == 0:
                print loss_cvr

        loss_cvr /= (dev-cev)
        print 'logloss: cvr = %.6f' % (loss_cvr)

#cv(911479, 1806748)
#cv(1806748, 2700288)

In [16]:
cv(911479, 1806748)
#cv(1806748, 2700288)

43094.5900569
86578.6765765
129660.615003
172849.97522
215990.068801
259387.115969
302631.694412
345830.252179
logloss: cvr = 0.432422
